# Informe de desempeño de PROESUR años 2015 - 2024

Este informe consta del desempeño del campus PROESUR desde el año 2015 hasta el año 2024. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
FOLDER = Path("data")  
OUTPUT = Path("Extracted_data")

In [3]:
def load_small_chunks(path: Path, col: str = "Cod_Estable",code: str = "05-02-0232-46") -> pd.DataFrame:
    '''
    Extracts only the data we want from each xlsx file 
    ''' 
    df = pd.read_excel(path, engine="openpyxl")
    df = df[df[col] == code]
    return df

def concat_dataframes(files: list[Path]) -> pd.DataFrame | None:
    df_list: list[pd.DataFrame] = []
    for f in files:
        df_list.append(load_small_chunks(f))
    try:
        result = pd.concat(df_list, ignore_index=True)
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None
 

In [4]:
def load_small_chunks_multiple(path: Path, col: str = "Cod_Estable", code: str = "05-02-0232-46") -> pd.DataFrame:
    """Read first sheet, keep ALL columns, filter by the first alias that exists."""
    df = pd.read_excel(path, engine="openpyxl", dtype=str)  # keep as strings
    try: 
        df = df[df[col] == code]  
    except Exception:
        df = df[df[col] == code]
        return df
    else:
        return df

    

In [ ]:

for f in sorted(FOLDER.glob("*.xlsx"), reverse=True):
    df = load_small_chunks_multiple(f)
    if not df.empty:
        df.to_csv(OUTPUT / f"{f.stem}_filtered.csv", index=False)
    else:
        print(f"{f.stem} was empty")


GRADUANDOS 2016 was empty
GRADUANDOS 2015 was empty
2019-Grad-Internet was empty
2018_-_Grad_Internet was empty
2017_Grad-Version_Internet was empty


In [5]:
def search_by_value(path: Path, val:str, col:str) -> pd.DataFrame:
    df = pd.read_excel(path, engine="openpyxl")
    mask = df[col].astype(str).str.contains(val, case=False, na=False)
    return df.loc[mask]


def search_by_map(path: Path, query: dict[str, str], op: str = "AND") -> pd.DataFrame:
    """
    query: {"NOM_ESTAB": "INSTITUTO", "Municipio": "Guatemala"}
    op: "AND" or "OR"
    """
    df = pd.read_excel(path, engine="openpyxl", dtype=str)
    masks = []
    for col, val in query.items():
        if col in df.columns:
            m = df[col].astype(str).str.contains(val, case=False, na=False)
            masks.append(m)
    if not masks:
        return df.iloc[0:0]
    if op.upper() == "OR":
        mask = masks[0]
        for m in masks[1:]: mask = mask | m
    else:
        mask = masks[0]
        for m in masks[1:]: mask = mask & m
    return df.loc[mask]


In [ ]:
FILES = ["GRADUANDOS 2015.xlsx", "GRADUANDOS 2016.xlsx"]

for fname in FILES:
    file_path = FOLDER / fname
    if not file_path.exists():
        print(f"File not found: {file_path}")
        continue
    result = search_by_value(file_path, "INSTITUTO TECNOLOGICO PROESUR ", "NOM_ESTAB")
    result.to_csv(OUTPUT / f"{fname}_filtered.csv", index=False)
    print(f"{fname}: {len(result)} rows found")

In [ ]:
FILES = ["2019-Grad-Internet.xlsx", "2018_-_Grad_Internet.xlsx", "2017_Grad-Version_Internet.xlsx"]
query = {"Cod_Muni": "0502", "Direc_Estable": "Km. 92.5 Finca Camantulul Carretera A Mazatenango"}
for fname in FILES:
    file_path = FOLDER / fname
    if not file_path.exists():
        print(f"File not found: {file_path}")
        continue
    result = search_by_map(file_path, query=query, op="AND")
    result.to_csv(OUTPUT / f"{fname}_filtered.csv", index=False)
    print(f"{fname}: {len(result)} rows found")

2019-Grad-Internet.xlsx: 68 rows found
2018_-_Grad_Internet.xlsx: 53 rows found
2017_Grad-Version_Internet.xlsx: 59 rows found


## Análisis de los datos obtenidos

In [ ]:

ALIASES = {

    # academic 
    "nivel_mate": [
        "DESEMPEÑO_MATEMÁTICAS"
    ],
    "nivel_lectura": [
        "DESEMPEÑO_LECTURA"
    ],
    "logro_mate": [
        "LOGRO_MATEMÁTICAS"
    ],
    "logro_rectura": [
        "LOGRO_LECTURA"
    ],
    "periodos_semanales_mate" : [
        "MAT_PERIODOS_MATEMATICA_SEMANA_Recodificada"
    ],
    "periodos_semanales_lectura" : [
        "LEC_PERIODOS_LECTURA_SEMANA_Recodificada"
    ],
    # socioeconomic 
    "trabaja": [
        "TRABAJA", "Trabajo_remunerado", "%Trabaja", "Trabaja_mientras_estudia"
    ],
    "acceso_tecnologia": [
        "ACCESO_TECNOLOGIA", "%AccesoTec", "Dispositivos", "Tecnologia"
    ],

}


In [18]:
CLEAN_PATH = Path("Extracted_data")
early_reports: list[str] = ["GRADUANDOS 2015.xlsx_filtered.csv", "GRADUANDOS 2016.xlsx_filtered.csv"]

exp_df = pd.read_csv(CLEAN_PATH / early_reports[0])
exp_df.iloc[:20, 190:]

,LC_LA_HORA,LC_PUBLINEWS,LC_OTRO,LEC_LIBROS_COMPLETOS_HA_LEIDO,LEC_LIBROS_COMPLETOS_HA_LEIDO_Recodificada,LEC_PERIODOS_LECTURA_SEMANA,LEC_PERIODOS_LECTURA_SEMANA_Recodificada,LEC_MINUTOS_DURA_PERIODO_LECTURA,LEC_MINUTOS_DURA_PERIODO_LECTURA_Recodificada,measure_MATE,DESEMPEÑO_MATEMÁTICAS,LOGRO_MATEMÁTICAS,measure_LECT,DESEMPEÑO_LECTURA,LOGRO_LECTURA
0,0,0,0,LIBROS_COMPLETOS_Ninguno,0,Periodo_Lect_6,6,MINUTOS_DURA_Lect_40_49,4,1.205,3.0,1.0,0.463,3.0,1.0
1,0,0,0,LIBROS_COMPLETOS_1_2,1,Periodo_Lect_6,6,MINUTOS_DURA_Lect_0_19,1,-0.420,1.0,0.0,0.561,3.0,1.0
2,0,0,0,LIBROS_COMPLETOS_1_2,1,Periodo_Lect_Ninguno,0,NaN,0,0.879,2.0,0.0,0.561,3.0,1.0
3,0,0,0,LIBROS_COMPLETOS_Ninguno,0,Periodo_Lect_6,6,MINUTOS_DURA_Lect_40_49,4,0.249,2.0,0.0,0.463,3.0,1.0
4,0,0,0,LIBROS_COMPLETOS_Mas6,4,Periodo_Lect_Ninguno,0,NaN,0,1.682,4.0,1.0,2.650,4.0,1.0
5,0,0,0,LIBROS_COMPLETOS_1_2,1,Periodo_Lect_6,6,MINUTOS_DURA_Lect_40_49,4,0.564,2.0,0.0,0.662,3.0,1.0
6,0,0,0,LIBROS_COMPLETOS_1_2,1,Periodo_Lect_6,6,MINUTOS_DURA_Lect_40_49,4,1.556,4.0,1.0,0.766,3.0,1.0
7,0,0,0,LIBROS_COMPLETOS_3_4,2,Periodo_Lect_Ninguno,0,NaN,0,1.436,4.0,1.0,0.766,3.0,1.0
8,0,0,0,LIBROS_COMPLETOS_Ninguno,0,Periodo_Lect_6,6,MINUTOS_DURA_Lect_40_49,4,1.084,3.0,1.0,0.409,2.0,0.0
9,0,0,0,LIBROS_COMPLETOS_Mas6,4,Periodo_Lect_6,6,MINUTOS_DURA_Lect_40_49,4,1.196,3.0,1.0,1.638,4.0,1.0
